In [4]:
import numpy as np
import os
import pandas as pd
from numpy.ma.extras import average

excel_path = r"C:\VISSIM_Workspace\network_new\test\network01_mer\vehicle_output"
df = pd.read_excel(os.path.join(excel_path, "1_vehicle_output.xlsx"))
df["NewMeasurem"] = df["Measurem."] % 1000

vehicle_type_list = [100, 300, 630, 640, 650]
results_list = []
for i in range(len(vehicle_type_list)):
    filtered_df = df[df["Vehicle type"] == vehicle_type_list[i]]
    # 피벗 테이블 생성
    old_df = filtered_df.pivot_table(
        index="time",
        columns=["NewMeasurem", "Measurem."],
        values="v[km/h]",
        aggfunc="mean"
    )

    # NewMeasurem 그룹화 후 같은 그룹, 같은 행에 있는 데이터의 평균 계산
    newmeasurem_avg = old_df.groupby(axis=1, level=0).mean()
    #print(newmeasurem_avg.head())
    # 평균값 (각 컬럼 기준)
    average_array = newmeasurem_avg.mean(axis=0)  # Series (index = NewMeasurem)

    # 표준편차 (각 컬럼 기준)
    std_array = newmeasurem_avg.std(axis=0) # Series (index = NewMeasurem)

    # 가중치: 평균 / 표준편차
    weights = average_array / std_array
    weights = np.nan_to_num(weights, nan=0.0, posinf=0.0, neginf=0.0)

    # 가중평균 계산 (각 컬럼 = 각 NewMeasurem 값별) : 평균속도 * 가중치 / 가중치(합)
    weighted_avg_array = ((average_array * weights) / weights).round(2)
    results_list.append(weighted_avg_array)
print(results_list)
# 리스트 → 데이터프레임 (행 방향으로)
results_df = pd.DataFrame(results_list, index=vehicle_type_list)

# 컬럼명은 NewMeasurem 값들, 인덱스는 Vehicle type
results_df.index.name = "Vehicle type"
results_df.to_excel(os.path.join(excel_path, "results.xlsx"))


[NewMeasurem
1      106.05
2      105.94
3      105.86
4      105.81
5      105.70
        ...  
116    103.46
117    103.49
118    103.51
119    103.46
120    103.46
Length: 119, dtype: float64, NewMeasurem
1      103.75
2      103.51
3      103.27
4      103.10
5      102.94
        ...  
116     92.31
117     92.33
118     92.35
119     92.34
120     92.34
Length: 119, dtype: float64, NewMeasurem
1      103.81
2      103.96
3      104.00
4      104.07
5      104.13
        ...  
116     98.82
117     98.69
118     98.50
119     98.14
120     98.05
Length: 119, dtype: float64, NewMeasurem
1      87.12
2      87.23
3      87.22
4      87.08
5      87.14
       ...  
116    85.76
117    85.32
118    85.32
119    85.62
120    85.79
Length: 119, dtype: float64, NewMeasurem
1      87.24
2      87.30
3      86.85
4      87.14
5      86.90
       ...  
116    86.94
117    87.69
118    87.03
119    86.87
120    86.71
Length: 119, dtype: float64]
